In [ ]:
# !pip install tensorflow==1.15.0

In [ ]:
# !pip install tensorflow-gpu==1.15.0

In [1]:
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
import h5py
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score
from collections import Counter
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


In [2]:
tf.__version__

'1.15.0'

In [3]:
random_seed = [1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 10000]

parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument("--dataname", default = "Quake_10x_Bladder", type = str)
parser.add_argument("--dataname", default = "data_-1.5c15", type = str)

parser.add_argument("--distribution", default = "ZINB")
parser.add_argument("--self_training", default = True)
parser.add_argument("--dims", default = [499, 256, 64, 32])
parser.add_argument("--highly_genes", default = 500)
parser.add_argument("--alpha", default = 0.001, type = float)
parser.add_argument("--gamma", default = 0.001, type = float)
parser.add_argument("--learning_rate", default = 0.0001, type = float)
parser.add_argument("--random_seed", default = random_seed)
parser.add_argument("--batch_size", default = 256, type = int)
parser.add_argument("--update_epoch", default = 10, type = int)
parser.add_argument("--pretrain_epoch", default = 1000, type = int)
parser.add_argument("--funetrain_epoch", default = 2000, type = int)
parser.add_argument("--t_alpha", default = 1.0)
parser.add_argument("--noise_sd", default = 1.5)
parser.add_argument("--error", default = 0.001, type = float)
parser.add_argument("--gpu_option", default = "0")

args = parser.parse_args("")

In [4]:
for category in [ "real_data",# "balanced_data", "imbalanced_data",
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)


    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal", "features"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue

        X = np.ceil(X).astype(np.int)
        count_X = X
        print(X.shape, count_X.shape)
        orig_X = X.copy()
        adata = sc.AnnData(X)
        adata.obs['Group'] = Y
        adata = normalize(adata,
                          copy=True,
                          highly_genes=args.highly_genes,
                          size_factors=True,
                          normalize_input=True,
                          logtrans_input=True)
        X = adata.X.astype(np.float32)
        Y = np.array(adata.obs["Group"])

        high_variable = np.array(adata.var.highly_variable.index, dtype=np.int)
        count_X = count_X[:, high_variable]
        size_factor = np.array(adata.obs.size_factors).reshape(-1,
                                                               1).astype(np.float32)
        cluster_number = int(max(Y) - min(Y) + 1)
        args.dims[0]=X.shape[1]
        print(X.shape, count_X.shape)

        for run in range(3):
            start = time.time()
            seed = run
            np.random.seed(seed)
            tf.reset_default_graph()
            chencluster = autoencoder(args.dataname, args.distribution,
                                      args.self_training, args.dims, cluster_number,
                                      args.t_alpha, args.alpha, args.gamma,
                                      args.learning_rate, args.noise_sd)

            chencluster.pretrain(X, count_X, size_factor, args.batch_size,
                                 args.pretrain_epoch, args.gpu_option)

            chencluster.funetrain(X, count_X, size_factor, args.batch_size,
                                  args.funetrain_epoch, args.update_epoch, args.error, Y)

            kmeans_ARI = np.around(adjusted_rand_score(Y, chencluster.kmeans_pred), 5)
            kmeans_NMI = np.around(
                normalized_mutual_info_score(Y, chencluster.kmeans_pred), 5)

            elapsed = time.time() - start
            ARI = adjusted_rand_score(Y, chencluster.Y_pred)
            NMI = np.around(normalized_mutual_info_score(Y, chencluster.Y_pred), 5)
            ss = silhouette_score(chencluster.latent_repre,chencluster.Y_pred)
            cal = calinski_harabasz_score(chencluster.latent_repre,chencluster.Y_pred)

            df.loc[df.shape[0]] = [dataset, ARI, NMI, ss, run, elapsed, chencluster.Y_pred, cal, 
                                   chencluster.latent_repre]
            df.to_pickle(f"../output/pickle_results/{category}/{category}_sczi.pkl")


['Quake_Smart-seq2_Trachea', 'Quake_Smart-seq2_Diaphragm', 'Quake_10x_Spleen', 'Young', 'mouse_ES_cell', 'Adam', 'Quake_10x_Bladder', 'Quake_Smart-seq2_Lung', 'Quake_10x_Limb_Muscle', 'worm_neuron_cell', 'mouse_bladder_cell', 'Romanov', 'Quake_Smart-seq2_Limb_Muscle', 'Muraro', '10X_PBMC']
>>>>dataset Quake_Smart-seq2_Trachea
(1350, 23341) (1350, 23341)
(1350, 500) (1350, 500)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

begin the pretraining





begin the funetraining
ARI 0.82762, NMI 0.74362
0 ARI 0.82762, NMI 0.74362
begin the pretraining
begin the funetraining
ARI 0.84394, NMI 0.73564
0 ARI 0.84394, NMI 0.73564
begin the pretraining
begin the funetraining
ARI 0.80593, NMI 0.71881
0 ARI 0.80593, NMI 0.71881
>>>>dataset Quake_Smart-seq2_Diaphragm
(870, 23341) (870, 23341)
(870, 499) (870, 499)
begin the pretraining
begin the funetraining
ARI 0.96107

(4186, 499) (4186, 499)
begin the pretraining
begin the funetraining
ARI 0.23573, NMI 0.39892
0 ARI 0.23577, NMI 0.39766
10 ARI 0.16317, NMI 0.38028
20 ARI 0.1062, NMI 0.31739
30 ARI 0.08097, NMI 0.27711
40 ARI 0.07816, NMI 0.27325
50 ARI 0.07641, NMI 0.27082
60 ARI 0.07487, NMI 0.26859
70 ARI 0.07392, NMI 0.26712
80 ARI 0.07326, NMI 0.2664
90 ARI 0.07259, NMI 0.26525
100 ARI 0.07251, NMI 0.26488
begin the pretraining
begin the funetraining
ARI 0.18423, NMI 0.35369
0 ARI 0.18441, NMI 0.35388
10 ARI 0.08361, NMI 0.28927
20 ARI 0.05511, NMI 0.21838
30 ARI 0.05018, NMI 0.20995
40 ARI 0.04466, NMI 0.20299
50 ARI 0.04334, NMI 0.20237
60 ARI 0.04369, NMI 0.20201
70 ARI 0.04366, NMI 0.20187
begin the pretraining
begin the funetraining
ARI 0.23702, NMI 0.40126
0 ARI 0.23767, NMI 0.40234
10 ARI 0.13182, NMI 0.3471
20 ARI 0.0851, NMI 0.28654
30 ARI 0.08049, NMI 0.28171
40 ARI 0.0776, NMI 0.27871
50 ARI 0.07703, NMI 0.27828
60 ARI 0.07673, NMI 0.278
70 ARI 0.07662, NMI 0.27793
80 ARI 0.07602, NMI

In [5]:
df.mean()

ARI       0.752219
NMI       0.780598
sil       0.256274
run       1.000000
time    152.106023
cal     596.273096
dtype: float64

In [ ]:
df.groupby("dataset").mean()